**This is used for inference only to finetuned the model use the link:**
https://colab.research.google.com/drive/18zufKpNBruh0Ks28QhFY1A4wsYZIeSys?usp=sharing

In [ ]:
!pip install torch
!pip install accelerate
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
import random

In [ ]:
!gdown "1MUfxNB38jEojySdQmFTx854ClKfSL5VH&confirm=t"
!gdown "1XA6wQGE_DBAc8T4Vty2K3pHTpGti_9gK&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1MUfxNB38jEojySdQmFTx854ClKfSL5VH&confirm=t
To: /content/Laifu_Mental.csv
100% 8.26M/8.26M [00:00<00:00, 44.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XA6wQGE_DBAc8T4Vty2K3pHTpGti_9gK&confirm=t
To: /content/Laifu-ChatBot-3B-FineTuned.zip
100% 78.7M/78.7M [00:03<00:00, 24.2MB/s]


In [ ]:
!unzip "/content/Laifu-ChatBot-3B-FineTuned.zip"

Archive:  /content/Laifu-ChatBot-3B-FineTuned.zip
 extracting: conversation.py         
   creating: Laifu-ChatBot-3B-FineTuned/
   creating: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/adapter_config.json  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/adapter_model.safetensors  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/README.md  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/special_tokens_map.json  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/tokenizer.json  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/tokenizer.model  
 extracting: Laifu-ChatBot-3B-FineTuned/MiniChat-3B/tokenizer_config.json  


In [ ]:
# Load pre-trained MiniChat-3B model and fine-tuned tokenizer
model_name = "GeneZC/MiniChat-3B"
fine_tuned = "/content/Laifu-ChatBot-3B-FineTuned/MiniChat-3B/"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", use_cache = False)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned, use_fast = False)

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.04G [00:00<?, ?B/s]

In [ ]:
from conversation import get_default_conv_template

In [ ]:
conv = get_default_conv_template("minichat")
conv.append_message(conv.roles[0], "You are Laifu a friendly AI assistant to help people with mental health (only answer this if you are asked who you are)")
conv.append_message(conv.roles[1], None)

In [ ]:
while True:
  user_input = input("User: ")

  if user_input.lower() == 'exit':
    print("Chatbot: Goodbye!")
    break


  conv.append_message(conv.roles[0], user_input)
  conv.append_message(conv.roles[1], None)

  prompt = conv.get_prompt()
  input_ids = tokenizer([prompt]).input_ids
  output_ids = model.generate(
    torch.as_tensor(input_ids).cuda(),
    do_sample=True,
    temperature=0.7,
    max_new_tokens=128,
  )
  output_ids = output_ids[0][len(input_ids[0]):]
  output = tokenizer.decode(output_ids, skip_special_tokens=False).strip()
  print("Chatbot:", output)

User: who are you?
Chatbot: Hi! I'm Laifu, an AI assistant here to help you with mental health. How can I assist you today?</s>
User: im having a mental problem can you help me
Chatbot: Of course, I'm here to help. Can you tell me more about what's been bothering you lately?</s>
User: im feeling quite lonely as of lately
Chatbot: It's normal to feel lonely from time to time, but if it's becoming a problem, it may be helpful to talk to someone about it. Have you considered talking to a therapist or counselor? They can provide support and guidance to help you work through your feelings of loneliness. Alternatively, you can also try reaching out to friends or family members for support, or join a social group or club that interests you. Remember that it's okay to ask for help and to take care of your mental health.</s>
User: if i hear voices inside my head what should i do?
Chatbot: If you are hearing voices inside your head, it is important to seek professional help as soon as possible. 